In [1]:
import warnings
# отключаем вывод предупреждений в консоль (чтобы она не засорялась)
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd
# библиотека для визуализации данных
import seaborn as sns 
import matplotlib.pyplot as plt
# библиотека для статистического анализа данных (статистических тестов)
from scipy import stats

# библиотека для градиентного бустинга
from xgboost import XGBClassifier
# библиотека для градиентного бустинга
from lightgbm import LGBMClassifier
# библиотека для градиентного бустинга и работы с категориальными данными
from catboost import CatBoostClassifier
# матрика для измерения качества моделей
from sklearn.metrics import roc_auc_score
# метод make_pipline создает конвейеры (объединяет шаги предобработки (кодирование признаков) и модель в единый процесс)
from sklearn.pipeline import make_pipeline
# кодирование категориальных признаков
from category_encoders import TargetEncoder
# оптимизация гиперпараметров или ансамблей
from hillclimbers import climb_hill, partial
# нейронная сеть - модель для классификации
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
# ансамблевый метод для классификации
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
# оценка калибровки вероятности модели
from sklearn.calibration import CalibrationDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# инструмент для кросс-валидации
from sklearn.model_selection import StratifiedKFold
# вычисление взаимной информации между признаками и целевой переменной
from sklearn.feature_selection import mutual_info_classif

In [2]:
oof = {}
test_pred = {}
NUM_FOLD = 5
target = 'loan_status'

In [124]:
train = pd.read_csv('data/train.csv', index_col='id')
test = pd.read_csv('data/test.csv', index_col='id')
original = pd.read_csv('data/credit_risk_dataset.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

In [125]:
original = original.reindex(columns=train.columns)

### EDA

In [119]:
train.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
id,,,,,,,,,,,,
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [6]:
test.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
id,,,,,,,,,,,
58645,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,N,2
58646,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,12.68,0.10,Y,4
58647,26,30000,RENT,5.0,VENTURE,E,4000,17.19,0.13,Y,2
58648,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,8.90,0.14,N,7
58649,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,16.32,0.15,Y,4


In [120]:
original.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,0.59,Y,3,1
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0.10,N,2,0
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,0.57,N,3,1
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,0.53,N,2,1
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,0.55,Y,4,1


### Проверим датасеты на наличие дубликатов

train и test

In [121]:
# Объединяем train и test, ищем дубликаты
merged = pd.concat([train.drop(columns=['loan_status']), test], axis=0)
duplicates = merged[merged.duplicated(keep=False)]  # keep=False помечает все дубликаты

print(f"Полных дубликатов между train и test: {len(duplicates)}")
print(f"Длина train: {len(train.values)}")
print(f"Длина test: {len(test.values)}")
print(f"Длина original: {len(original.values)}")

Полных дубликатов между train и test: 0
Длина train: 58645
Длина test: 39098
Длина original: 32581


train и original

In [122]:
merged = pd.concat([train, original], axis=0)
duplicates1 = merged[merged.duplicated(keep=False)]  # keep=False помечает все дубликаты

print(f"Полных дубликатов между train и original: {len(duplicates1)}")
print(f"Длина train: {len(train.values)}")
print(f"Длина test: {len(test.values)}")
print(f"Длина original: {len(original.values)}")

Полных дубликатов между train и original: 330
Длина train: 58645
Длина test: 39098
Длина original: 32581


In [ ]:
original_copy = original.copy()
for i in range(len(original.values)):
    if (train == original.values[i]).all(axis=1).any():
        original_copy.drop(i, axis=0, inplace=True)

print(len(original_copy))

test и original

In [25]:
merged = pd.concat([test, original.drop(columns='loan_status')], axis=0)
duplicates2 = merged[merged.duplicated(keep=False)]  # keep=False помечает все дубликаты

print(f"Полных дубликатов между test и original: {len(duplicates2)}")
print(f"Длина train: {len(train.values)}")
print(f"Длина test: {len(test.values)}")
print(f"Длина original: {len(original.values)}")

Полных дубликатов между test и original: 332
